# Compare OSM2GMNS Output to OSMnx and SharedStreets

In [20]:
import geopandas as gpd
import pandas as pd
import pyarrow.feather as arrow

from shapely import wkt
from geopandas.tools import sjoin

## Remote I/O

In [21]:
gmns_links_filename = "link.csv"
gmns_nodes_filename = "node.csv"
boundary_filename = "nw_pdx.geojson"
shst_filename = "shst.feather"
osmnx_filename = "osmnx_link_disk.geojson"

## Data Reads

In [22]:
osmnx_gdf = gpd.read_file(osmnx_filename)
link_df = pd.read_csv(gmns_links_filename)
node_df = pd.read_csv(gmns_nodes_filename)
nw_pdx_gdf = gpd.read_file(boundary_filename)
shst_df = arrow.read_feather(shst_filename)

In [23]:
join_node_df = node_df[["node_id", "osm_node_id"]]

In [24]:
df = link_df.copy()
df["geometry"] = df.rename(columns = {"geometry": "geo_string"}).geo_string.apply(wkt.loads)

link_gdf = gpd.GeoDataFrame(df, geometry="geometry", crs=4326)
link_gdf = link_gdf[["link_id", "name", "osm_way_id", "from_node_id", "to_node_id", "geometry"]]

link_gdf = pd.merge(
    link_gdf,
    join_node_df,
    how="left",
    left_on="from_node_id",
    right_on="node_id",
).rename(columns={"osm_node_id":"u"}).drop("node_id", axis=1)

link_gdf = pd.merge(
    link_gdf,
    join_node_df,
    how="left",
    left_on="to_node_id",
    right_on="node_id",
).rename(columns={"osm_node_id":"v"}).drop("node_id", axis=1)

link_gdf = sjoin(link_gdf, nw_pdx_gdf, how="left")
link_gdf = link_gdf[link_gdf["id"].notnull()]
link_gdf = link_gdf.drop(["index_right", "id"], axis=1)

link_gdf.head()

,link_id,name,osm_way_id,from_node_id,to_node_id,geometry,u,v
34,34,Northwest Torreyview Drive,5386342,31,32,"LINESTRING (-122.76859 45.52197, -122.76890 45...",38704639.0,38704643.0
35,35,Northwest Torreyview Drive,5386342,32,31,"LINESTRING (-122.76905 45.52206, -122.76890 45...",38704643.0,38704639.0
42,42,Northwest 95th Avenue,5386696,39,40,"LINESTRING (-122.77542 45.52142, -122.77553 45...",38711211.0,38711213.0
43,43,Northwest 95th Avenue,5386696,40,39,"LINESTRING (-122.77602 45.52148, -122.77591 45...",38711213.0,38711211.0
44,44,NaN,5386821,41,42,"LINESTRING (-122.77548 45.52096, -122.77533 45...",38713215.0,38713221.0


In [25]:
osmnx_gdf.head()

,osmid,u,v,geometry
0,None,1367343800,40541098,"LINESTRING (-122.66808 45.55078, -122.66848 45..."
1,None,1367343800,7990710833,"LINESTRING (-122.66808 45.55078, -122.66807 45..."
2,None,1367343801,7990710837,"LINESTRING (-122.66784 45.55079, -122.66785 45..."
3,None,1367343802,40675211,"LINESTRING (-122.66794 45.55086, -122.66794 45..."
4,None,1367343802,7990710834,"LINESTRING (-122.66794 45.55086, -122.66787 45..."


In [26]:
osmnx_df = pd.DataFrame(osmnx_gdf[["u", "v"]])
osmnx_df.insert(0, "osmnx_id", range(1, 1 + len(osmnx_df)))
osmnx_df.head()

,osmnx_id,u,v
0,1,1367343800,40541098
1,2,1367343800,7990710833
2,3,1367343801,7990710837
3,4,1367343802,40675211
4,5,1367343802,7990710834


In [27]:
output_gdf = pd.merge(
    link_gdf,
    osmnx_df,
    how="left",
    on=["u","v"]
)

output_gdf = pd.merge(
    output_gdf,
    shst_df,
    how="left",
    on=["u","v"],
)

output_gdf.head()

,link_id,name,osm_way_id,from_node_id,to_node_id,geometry,u,v,osmnx_id,shst_geometry_id
0,34,Northwest Torreyview Drive,5386342,31,32,"LINESTRING (-122.76859 45.52197, -122.76890 45...",38704639.0,38704643.0,NaN,13c4f787ffa1d64e57b29082ea92b017
1,35,Northwest Torreyview Drive,5386342,32,31,"LINESTRING (-122.76905 45.52206, -122.76890 45...",38704643.0,38704639.0,NaN,NaN
2,42,Northwest 95th Avenue,5386696,39,40,"LINESTRING (-122.77542 45.52142, -122.77553 45...",38711211.0,38711213.0,NaN,eab045284d4aaabe813dad9ba243841b
3,43,Northwest 95th Avenue,5386696,40,39,"LINESTRING (-122.77602 45.52148, -122.77591 45...",38711213.0,38711211.0,NaN,NaN
4,44,NaN,5386821,41,42,"LINESTRING (-122.77548 45.52096, -122.77533 45...",38713215.0,38713221.0,NaN,NaN


In [28]:
output_gdf.to_file("compare_osm2gmns.geojson", driver='GeoJSON')